# Getting Started with PyAERMOD

This notebook demonstrates the basic PyAERMOD workflow:

1. Generate AERMOD input files from Python
2. Run AERMOD (if executable available)
3. Parse output files
4. Analyze and visualize results

**Requirements:**
- PyAERMOD installed (`pip install -e .`)
- AERMOD executable (optional for this demo)
- Meteorological data files (optional for this demo)

## 1. Installation Check

First, let's verify PyAERMOD is installed correctly:

In [ ]:
# Import PyAERMOD modules
try:
    from pyaermod.input_generator import (
        ControlPathway, PollutantType, TerrainType,
        SourcePathway, PointSource,
        ReceptorPathway, CartesianGrid,
        MeteorologyPathway, OutputPathway,
        AERMODProject
    )
    from pyaermod.output_parser import parse_aermod_output
    from pyaermod.runner import AERMODRunner
    
    print("✓ PyAERMOD imported successfully!")
except ImportError as e:
    print(f"✗ Error importing PyAERMOD: {e}")
    print("\nPlease install required modules")

## 2. Create a Simple Point Source Model

Let's create a basic AERMOD model with a single point source (stack).

In [ ]:
# Define control parameters
control = ControlPathway(
    title_one="My First AERMOD Model",
    title_two="Single point source example",
    pollutant_id=PollutantType.PM25,
    averaging_periods=["ANNUAL", "24"],
    terrain_type=TerrainType.FLAT
)

print("✓ Control pathway defined")
print(f"  Pollutant: {control.pollutant_id}")
print(f"  Averaging periods: {control.averaging_periods}")
print(f"  Terrain: {control.terrain_type}")

In [ ]:
# Define emission source
sources = SourcePathway()

stack = PointSource(
    source_id="STACK1",
    x_coord=0.0,
    y_coord=0.0,
    base_elevation=100.0,  # meters
    stack_height=50.0,     # meters
    stack_temp=400.0,      # Kelvin
    exit_velocity=15.0,    # m/s
    stack_diameter=2.0,    # meters
    emission_rate=1.5      # g/s
)

sources.add_source(stack)

print("✓ Source defined")
print(f"  Source ID: {stack.source_id}")
print(f"  Location: ({stack.x_coord}, {stack.y_coord})")
print(f"  Stack height: {stack.stack_height} m")
print(f"  Emission rate: {stack.emission_rate} g/s")

In [ ]:
# Define receptor grid
receptors = ReceptorPathway()

grid = CartesianGrid.from_bounds(
    x_min=-1000.0,
    x_max=1000.0,
    y_min=-1000.0,
    y_max=1000.0,
    spacing=100.0  # 100m grid spacing
)

receptors.add_cartesian_grid(grid)

print("✓ Receptor grid defined")
print(f"  Grid size: {grid.x_num} x {grid.y_num} = {grid.x_num * grid.y_num} receptors")
print(f"  Spacing: {grid.x_delta} m")

In [ ]:
# Define meteorology
meteorology = MeteorologyPathway(
    surface_file="example_met.sfc",
    profile_file="example_met.pfl"
)

print("✓ Meteorology defined")
print(f"  Surface file: {meteorology.surface_file}")
print(f"  Profile file: {meteorology.profile_file}")

In [ ]:
# Define output options
output = OutputPathway(
    receptor_table=True,
    receptor_table_rank=10,
    max_table=True,
    summary_file="getting_started.sum"
)

print("✓ Output options defined")

## 3. Create Project and Generate Input File

Now we combine everything into a project and write the AERMOD input file:

In [ ]:
# Create project
project = AERMODProject(
    control=control,
    sources=sources,
    receptors=receptors,
    meteorology=meteorology,
    output=output
)

# Write input file
input_file = "getting_started.inp"
project.write(input_file)

print(f"✓ Input file created: {input_file}")

In [ ]:
# Let's look at the generated input file
print("Generated AERMOD input file:")
print("="*70)
print(project.to_aermod_input())
print("="*70)

## 4. Run AERMOD (Optional)

If you have AERMOD installed, you can run it directly from Python:

In [ ]:
# Try to run AERMOD (skip if not installed)
try:
    from pyaermod import AERMODRunner
    
    runner = AERMODRunner()
    result = runner.run(input_file, timeout=600)
    
    if result.success:
        print(f"✓ AERMOD completed successfully")
        print(f"  Runtime: {result.runtime_seconds:.1f} seconds")
        print(f"  Output file: {result.output_file}")
    else:
        print(f"✗ AERMOD failed: {result.error_message}")
        
except FileNotFoundError:
    print("ℹ AERMOD executable not found in PATH")
    print("  To run AERMOD, install it from EPA SCRAM website")
    print("  For now, we'll use sample output for demonstration")

## 5. Parse Output (Demo with Sample Data)

For this demo, we'll create a sample output to show how parsing works:

In [ ]:
# Create sample output for demonstration
sample_output = """
*** AERMOD - VERSION  24142 ***   *** Jobname: Getting_Started               ***

 *** ANNUAL RESULTS ***

   X-COORD       Y-COORD    CONC (UG/M3)
      0.00         0.00         5.2341
    100.00         0.00         8.4123
    200.00         0.00        12.6789
    300.00         0.00        15.2341
    400.00         0.00        18.9876
"""

with open('getting_started_sample.out', 'w') as f:
    f.write(sample_output)

print("✓ Sample output file created for demonstration")

In [ ]:
# Parse the output
from pyaermod import parse_aermod_output

results = parse_aermod_output('getting_started_sample.out')

print("✓ Output parsed successfully")
print("\nRun Information:")
print(f"  Version: {results.run_info.version}")
print(f"  Job name: {results.run_info.jobname}")
print(f"  Pollutant: {results.run_info.pollutant_id}")

## 6. Analyze Results

PyAERMOD converts results to pandas DataFrames for easy analysis:

In [ ]:
# Get concentration data as DataFrame
annual_df = results.get_concentrations('ANNUAL')

print("Annual Concentrations:")
print(annual_df)

# Basic statistics
print("\nStatistics:")
print(f"  Mean: {annual_df['concentration'].mean():.4f} ug/m³")
print(f"  Max:  {annual_df['concentration'].max():.4f} ug/m³")
print(f"  Min:  {annual_df['concentration'].min():.4f} ug/m³")

In [ ]:
# Find maximum concentration location
max_info = results.get_max_concentration('ANNUAL')

print("\nMaximum Concentration:")
print(f"  Value: {max_info['value']:.4f} ug/m³")
print(f"  Location: ({max_info['x']:.2f}, {max_info['y']:.2f})")
print(f"  Units: {max_info['units']}")

## 7. Compliance Checking

Check against regulatory standards:

In [ ]:
# PM2.5 NAAQS (National Ambient Air Quality Standard)
pm25_annual_standard = 12.0  # ug/m³

max_conc = max_info['value']

print("\nCompliance Check:")
print(f"  Standard: {pm25_annual_standard} ug/m³ (PM2.5 Annual)")
print(f"  Maximum:  {max_conc:.2f} ug/m³")
print(f"  Percentage: {(max_conc/pm25_annual_standard)*100:.1f}% of standard")

if max_conc > pm25_annual_standard:
    print(f"  Status: ⚠️  EXCEEDS STANDARD")
else:
    print(f"  Status: ✅ COMPLIES")

## 8. Export Results

Export to CSV for further analysis:

In [ ]:
# Export to CSV
results.export_to_csv("results", prefix="getting_started")

print("✓ Results exported to CSV files in 'results/' directory")

## Summary

In this notebook, you learned how to:

1. **Define model parameters** - Control, sources, receptors, meteorology
2. **Generate AERMOD input** - Create `.inp` files from Python
3. **Run AERMOD** - Execute from Python (optional)
4. **Parse output** - Convert to pandas DataFrames
5. **Analyze results** - Statistical analysis
6. **Check compliance** - Compare against standards
7. **Export data** - Save to CSV

### Next Steps

- **Notebook 02**: Point Source Modeling (detailed)
- **Notebook 03**: Area Source Modeling
- **Notebook 04**: Parameter Sweeps
- **Notebook 05**: Visualization
- **Notebook 06**: POSTFILE Analysis — parse binary/text POSTFILE output, extract deposition data, time-series at specific receptors
- **Notebook 07**: Advanced Features — background concentrations, deposition modeling, EVENT processing, NO2 chemistry, source groups

### Parsing POSTFILE Output

After running AERMOD with POSTFILE output enabled, you can parse the results directly:

```python
from pyaermod.postfile import read_postfile

# Auto-detects text (PLOT) vs binary (UNFORM) format
result = read_postfile("postfile.bin")

# Access timestep data as a DataFrame
df = result.data
print(df.head())

# With deposition columns (concentration + dry + wet)
result = read_postfile("postfile.bin", has_deposition=True)
print(result.data.columns)
# ['timestep', 'receptor', 'concentration', 'dry_depo', 'wet_depo']
```

### Resources

- [PyAERMOD Documentation](https://github.com/atmmod/pyaermod)
- [AERMOD User Guide](../reference/aermod_userguide.pdf)
- [Quick Reference](../reference/aermod_quick_reference.pdf)